In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import re

In [ ]:
num = 1
while True:
    main_url = f'https://uppity.co.kr/column/moneylog/{num}/'
    response = requests.get(main_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # <div class="uc_post_title"> 안에 있는 a 태그의 href 링크 추출
    uc_post_titles = soup.find_all('div', class_='uc_post_title')

    # 만약 페이지에 값이 없으면 반복을 종료
    if not uc_post_titles:
        print(f"Page {num} is empty, stopping the loop.")
        break

    hrefs = [post_title.find('a')['href'] for post_title in uc_post_titles if post_title.find('a')]

    # 각 href에 해당하는 페이지로 들어가 div 값 수집
    for href in hrefs:
        post_response = requests.get(href)  # 추출한 href로 들어가기
        post_soup = BeautifulSoup(post_response.text, 'html.parser')

        # 지정된 div 요소 찾기
        target_div = post_soup.select_one('div[data-id="31f5ad4"][data-element_type="container"].elementor-element.elementor-element-31f5ad4.e-flex.e-con-boxed.e-con.e-child div.e-con-inner')

        # 해당 div에서 텍스트 추출
        if target_div:
            content = target_div.get_text(separator="\n").strip()
        else:
            content = "지정된 div를 찾을 수 없습니다."

        # 데이터를 리스트에 추가
        data.append({
            'url': href,
            'div': content
        })

    # 다음 페이지로 넘어가기 위해 num 값을 증가
    num += 1

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['url', 'div'])

Page 19 is empty, stopping the loop.


In [ ]:
print(df.shape)
df.drop_duplicates(subset='div',keep='first',inplace=True)
print(df.shape)

(420, 9)
(235, 9)


In [ ]:
def remove_before_money_profile(text):
    money_profile_pos = text.find("머니 프로필")
    if money_profile_pos != -1:
        return text[money_profile_pos:]  # "머니 프로필" 이후의 데이터만 반환
    return text  # "머니 프로필"이 없으면 원본 텍스트 반환

# 데이터프레임에서 'div' 컬럼의 데이터를 파싱하여 "머니 프로필" 이전 데이터를 삭제
df['div'] = df['div'].apply(remove_before_money_profile)

In [ ]:
def extract_value(text, label):
    pattern = rf"{label}:\s*\n*(.+?)\n"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

def extract_property(text):
    pattern = r"현재 자산\s*\n(.+?)\n나의 한 달 현금 흐름 관리 방법"
    match = re.search(pattern, text, re.DOTALL)  # DOTALL 플래그로 여러 줄에 걸친 데이터를 추출
    return match.group(1).strip() if match else None

def extract_moeny(text):
    pattern = r"월평균 고정비\s*\n*(.+?)\n.*?[^\S\n]*머니로그"
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)  # DOTALL 플래그로 여러 줄에 걸친 데이터를 추출, IGNORECASE로 대소문자 무시
    return match.group(1).strip() if match else None


def extract_consult(text):
    pattern = r"돈 관련 고민과 어피티의 솔루션\s*(.+)"  # 이후 모든 데이터를 추출하는 패턴
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)  # DOTALL 플래그로 여러 줄에 걸친 데이터를 추출, IGNORECASE로 대소문자 무시
    return match.group(1).strip() if match else None


def extract_property(text):
    # 1. 먼저 "머니로그"가 포함된 문구를 찾는 임시 변수 temp
    temp = find_moneylog_phrase(text)

    # 2. "머니로그" 문구가 있으면 그 이전의 "현재 자산"부터 추출
    if temp:
        pattern = rf"현재 자산\s*\n(.+?)\n{re.escape(temp)}"  # temp 값 사용, 정규 표현식으로 처리
        match = re.search(pattern, text, re.DOTALL)
        return match.group(1).strip() if match else None

    return None  # "머니로그" 문구가 없으면 None 반환

# "머니로그"가 포함된 문구를 찾고 저장하는 함수 정의
def find_moneylog_phrase(text):
    pattern = r"\n.*머니로그.*\n"  # "머니로그"가 포함된 문구를 찾는 정규 표현식
    match = re.search(pattern, text)
    return match.group(0).strip() if match else None

# 파싱 작업을 수행하여 각 컬럼을 추가
df['age'] = df['div'].apply(lambda x: extract_value(x, '나이'))
df['goal'] = df['div'].apply(lambda x: extract_value(x, '돈 관련 목표'))
df['job'] = df['div'].apply(lambda x: extract_value(x, '하는 일'))
df['salary'] = df['div'].apply(lambda x: extract_value(x, '세전연봉'))
df['mon_sal'] = df['div'].apply(lambda x: extract_value(x, '월 평균 실수령액'))
df['house'] = df['div'].apply(lambda x: extract_value(x, '주거형태'))
df['property'] = df['div'].apply(extract_property)
df['solution'] = df['div'].apply(extract_consult)

In [ ]:
df.head(3)

,url,div,age,goal,job,salary,mon_sal,house,property,solution
0,https://uppity.co.kr/%ec%9b%94-%eb%82%a9%ec%9e...,머니 프로필\n닉네임: \n일이칠\n나이: \n만 25\n금융성향\n:\n 현실적인...,만 25,내년 해외 이직을 위해 적어도 미국에서 6개월 이상 여유롭게 지낼 수 있도록 노는 ...,비영리단체 해외파견직(PAO),3420만 원,"270만+α(부수입인 책 리뷰 포스팅, 외주 작업, 선교 헌금으로 들어오는 용돈)",공동 생활(주거비 안 듦),비상금: \n1350만 원\n예적금: \n340만 원\n외화 예금: \n600달러\...,"Q. 미국 생활 전에 자산 전략이 필요해요 \n사회생활 극초반에, 그간 저금해 놓..."
1,https://uppity.co.kr/%ed%88%ac%ec%9e%90%eb%8f%...,머니 프로필\n닉네임: \n비비비\n나이: \n만 34세\n금융성향\n:\n 현실적...,만 34세,소비 줄여서 시드머니 불리기,소프트웨어 개발팀 본부장,1억4500만 원,"910만 원, 월세 수입 250만 원",전세,비상금: \n2600만 원\n예·적금: \n5000만 원\n보증금: \n1억\n대출...,"Q. 주식, 생활비, 부동산에 기준이 필요해요 \n이것저것 너무 많은 것들을 하다 ..."
2,https://uppity.co.kr/%ec%b9%b4%eb%93%9c%ea%b0%...,머니 프로필\n닉네임: \n너무 놀아서 현타오는 인생\n나이: \n만 28세\n금융...,만 28세,31살 전에 1억 모으기,제약회사 연구원,None,350만 원,본가 거주,"예적금:\n 1260만 원(희망적금 만기 후 도약계좌), 1500만 원(예금), 1...",Q. 어디서부터 시작해야 할까요?\n일한 지 5년 차지만 쇼핑과 노는 걸 좋아하다보...


In [ ]:
df.to_csv('/content/drive/MyDrive/rag.csv')